In [153]:
#!pip install py2neo
from py2neo import Graph
import pandas as pd

In [154]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

In [155]:
type(graph)

py2neo.database.Graph

# Importing Data

In [156]:
dfstudent = pd.read_csv('g3dbstudent.csv')
dfcourses = pd.read_csv('g3dbcourses.csv')
dfsection = pd.read_csv('g3dbsection.csv')
dfdpt = pd.read_csv('g3dbdpt.csv')
dftakingtaken = pd.read_csv('g3dbtaking.csv')
dfclub = pd.read_csv('g3dbclub.csv')
dfclubrole = pd.read_csv('g3dbcrole.csv')

In [60]:
#Dropping 1st row in Students
dfstudent1 = dfstudent.drop(index=dfstudent.index[0], axis=0)
dfstudent1


,student_id,first_name,last_name,dept_code,total_credits,student_year,living_status
1,1,John,Doe,ECON,100,Senior,Off-campus
2,2,Jon,Kashmeeri,BUS,113,Senior,Off-campus
3,3,Mary,Jane,NUTR,25,Freshman,On-campus
4,4,Mark,Maguire,CS,95,Senior,On-campus
5,5,Jasmine,Wang,PSYC,23,Freshman,On-campus
6,6,Barbara,Olsen,MUSI,66,Junior,On-campus
7,7,Johnny,Depp,STAT,85,Junior,Off-Campus
8,8,Jose,Pein,CS,15,Freshman,Off-Campus
9,9,Bo,Orblian,CS,35,Sophomore,Off-Campus
10,10,Forge,Launch,BIOL,24,Freshman,On-campus


In [63]:
#Dropping 1st row in Dpt
dfdpt1 = dfdpt.drop(index=dfstudent.index[0], axis=0)
dfdpt1

,dept_code,dept_name,num_staff,num_students,budget
1,BIOL,Biology,108,223,230000
2,BUS,Business,45,143,200000
3,CHEM,Chemistry,25,86,120000
4,CS,Computer Science,40,170,190000
5,ECON,Economics,50,150,200000
6,ENGL,English,24,204,16000
7,HIST,History,12,50,60000
8,IT,Information Technology,46,200,200000
9,LFIT,Fitness,16,245,18000
10,MATH,Mathematics,20,101,120000


# Creating Nodes and Relationships

In [210]:
#Creating Departments
for i in range(len(dfdpt1)):
    row = dfdpt1.iloc[i]
    graph.run("CREATE (:DEPARTMENT {dept_code:'" + row['dept_code'] + "',dept_name:'" + row['dept_name'] + "',num_staff:" + row['num_staff'] + ",num_students:" + row['num_students'] + ",budget:" + row['budget'] + "})")
    

In [211]:
#Creating Students
for i in range(len(dfstudent1)):
    row = dfstudent1.iloc[i]
    #Create the person nodes, with the properties that won't be used to create relationships.
    graph.run("CREATE (:STUDENT {student_id:'" + row['student_id'] + "', first_name:'" + row['first_name'] + "',last_name:'" + row['last_name'] + "',dept_code:'" + row['dept_code'] + "',total_credits:" + str(row.total_credits) + ",student_year:'" + row['student_year'] + "',living_status:'" + row['living_status'] + "'})")
    
    #Relationship between student and department
    graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] + "', first_name:'" + row['first_name'] + "',last_name:'" + row['last_name'] + "',total_credits:" + str(row.total_credits) + ",student_year:'" + row['student_year'] + "',living_status:'" + row['living_status'] + "'}), (d:DEPARTMENT {dept_code:'" + row['dept_code'] + "'}) CREATE (s)-[:MAJORS_IN]->(d)")
    

In [213]:
#Creating Clubs #realizing that clubid should be another column in students
#dfclub.drop(index=dfclub.index[0], axis=0, inplace=True)
for i in range(len(dfclub)):
    row = dfclub.iloc[i]
    graph.run("CREATE (:CLUBS {club_id:'" + row['club_id'] + "', club_name:'" + row['club_name'] + "',description:'" + row['description ( ? )'] + "',num_members:" + row['num_members'] + "})")


In [214]:
#dfclubrole.drop(index=dfclubrole.index[0], axis=0, inplace=True)
for i in range(len(dfclubrole)):
    row = dfclubrole.iloc[i]
    #graph.run("CREATE (:CLUB_ROLE {role:'" + row['role'] + "'})")
#matching
    graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] + "'}), (c:CLUBS {club_id:'" + row['club_id'] + "'}) CREATE (s)-[:IS_IN{role:'" + row['role'] + "'}]->(c)")

In [175]:
#dfcourses.drop(index=dfcourses.index[0], axis=0, inplace=True)
dfcourses

,course_id,course_title,dept_code,credits
1,BIOL101,Principles of Biology,BIOL,3
2,BIOL101L,Intro Biology Laboratory,BIOL,1
3,BIOL252,Anatomy and Physiology,BIOL,3
4,CHEM261,Analytical Chemistry,CHEM,4
5,CHEM241,Organic Chemistry,CHEM,4
6,CS310,Data Structures,CS,3
7,CS126,Intro to Web Design and Development,CS,3
8,CS110,Intro to Data Science,CS,3
9,CS116,Intro to Scientific Programming,CS,3
10,ECON125,Intro to Entrepreneurship,ECON,3


In [215]:
#Creating Courses
for i in range(len(dfcourses)):
    row = dfcourses.iloc[i]
    graph.run("CREATE (:COURSES {course_id:'" + row['course_id'] + "',dept_code:'" + row['dept_code'] + "', course_title:'" + row['course_title'] + "',credits:" + row['credits'] + "})")
    

In [216]:
#dftakingtaken.drop(index=dftakingtaken.index[0], axis=0, inplace=True)
#dftakingtaken
for i in range(len(dftakingtaken)):
    row = dftakingtaken.iloc[i]
    graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] + "'}), (c:COURSES {course_id:'" + row['course_id'] + "'}) CREATE (s)-[:TAKEN_OR_TAKING {section_id:'" + row['section_id'] + "', semester:'" + row['semester'] + "',year:'" + row['year'] + "'}]->(c)")


In [208]:
#Creating Sections #just grouping by building #issues with duplicates #notdoing
#dfsection.drop(index=dfsection.index[0], axis=0, inplace=True)
dfsection
for i in range(len(dfsection)):
    row = dfsection.iloc[i]
    graph.run("CREATE (:SECTION {building:'" + row['building'] + "'})")
    graph.run("MATCH (c:COURSES {course_id:'" + row['course_id'] + "'}), (s:SECTION {building:'" + row['building'] + "'}) CREATE (c)-[:HELD_AT {max_capacity:'" + row['max_capacity'] + "', num_registered:'" + row['num_registered'] + "', room_num:'" + row['room_num'] + "',time_slot:'" + row['time_slot'] + "'}]->(s)")
    

DatabaseError: [Schema.ConstraintCreationFailed] Unable to create Constraint( name='constraint_cffff8bb', type='UNIQUENESS', schema=(:SECTION {building}) ):
Both Node(27) and Node(28) have the label `SECTION` and property `building` = 'Aquia'

# Deleting Nodes/Relationships to Rerun

In [209]:
#graph.run("Match (s:STUDENT)-[r:HAS_TAKEN_IS_TAKING]->() DELETE r")
#graph.run("Match (s:STUDENT)-[]->(c:COURSES) DETACH DELETE c")
#graph.run("Match (s:STUDENT)-[]->(d:DEPARTMENT) DETACH DELETE s")
graph.run("Match (n) Detach Delete n")

(No data)

# Running Queries

In [168]:
#finding presidents
    print("MATCH (s:STUDENT)-[r:IS_IN {role: 'President'}]->(c:CLUBS) RETURN s,c")

MATCH (s:STUDENT)-[r:IS_IN {role: 'President}]->(c:CLUBS) RETURN s,c
MATCH (s:STUDENT)-[r:IS_IN {role: 'President}]->(c:CLUBS) RETURN s,c
MATCH (s:STUDENT)-[r:IS_IN {role: 'President}]->(c:CLUBS) RETURN s,c
MATCH (s:STUDENT)-[r:IS_IN {role: 'President}]->(c:CLUBS) RETURN s,c
MATCH (s:STUDENT)-[r:IS_IN {role: 'President}]->(c:CLUBS) RETURN s,c
MATCH (s:STUDENT)-[r:IS_IN {role: 'President}]->(c:CLUBS) RETURN s,c
MATCH (s:STUDENT)-[r:IS_IN {role: 'President}]->(c:CLUBS) RETURN s,c
MATCH (s:STUDENT)-[r:IS_IN {role: 'President}]->(c:CLUBS) RETURN s,c
MATCH (s:STUDENT)-[r:IS_IN {role: 'President}]->(c:CLUBS) RETURN s,c
MATCH (s:STUDENT)-[r:IS_IN {role: 'President}]->(c:CLUBS) RETURN s,c
MATCH (s:STUDENT)-[r:IS_IN {role: 'President}]->(c:CLUBS) RETURN s,c
MATCH (s:STUDENT)-[r:IS_IN {role: 'President}]->(c:CLUBS) RETURN s,c
MATCH (s:STUDENT)-[r:IS_IN {role: 'President}]->(c:CLUBS) RETURN s,c
MATCH (s:STUDENT)-[r:IS_IN {role: 'President}]->(c:CLUBS) RETURN s,c
MATCH (s:STUDENT)-[r:IS_IN {role: 

In [188]:
#finding students taking classes in their major

#initial try
#for i in range(len(dfdpt1)):
    #row = dfdpt1.iloc[i]
   #graph.run("MATCH (s:STUDENT {dept_code:'" + row['dept_code'] + "'})-[]->(c:COURSES {dept_code:'" + row['dept_code'] + "'}) RETURN s")
#what worked in Neo4j ==> MATCH (s:STUDENT)-[]->(c:COURSES{dept_code:s.dept_code} ) RETURN s,c


In [ ]:
#finding proportion of intro courses
#using GUI
#MATCH (c:COURSES) WHERE c.course_title STARTS WITH 'Intro' RETURN c